# Banco de dados Solo Criado: Take 1

Vou fazer primeiro no notebook para ir testando, depois passo para o arquivo .py

## Roadmap

* Coletar listagem do site da prefeitura
* Transformar pdf da listagem em xmls por este site
* Transformar xmls em cvs
* Carregar cvs de listagem para o banco de dados
  * Extração;
  * Exploração;
  * Limpeza;
  * Agregação;
  * Visualização;
  * Storytelling.

## Primeira parte: Coleta e transformação dos dados

from [here](https://www.edureka.co/blog/web-scraping-with-python/): <br>
Libraries used for Web Scraping  <br>
As we know, Python is has various applications and there are different libraries for different purposes. In our further demonstration, we will be using the following libraries: <br>
<br>
**Selenium**:  Selenium is a web testing library. It is used to automate browser activities. <br>
**BeautifulSoup**: Beautiful Soup is a Python package for parsing HTML and XML documents. It creates parse trees that is helpful to extract the data easily.<br>
Pandas: Pandas is a library used for data manipulation and analysis. It is used to extract the data and store it in the desired format. 

Eu tenho que baixar com requests somente a parte que me interessa (div id="conteudo" class="outros") <br>
Ou usar o bs4 pra retirar somente essa parte do todo

>import requests as rq
>
>url = 'http://www2.portoalegre.rs.gov.br/edificapoa/default.php?p_secao=1445'
>r = rq.get(url, allow_redirects=True)
>open('estoques.html', 'wb').write(r.content)

Se eu conseguir baixar a pagina, posso usar o BeautifulSoup para separar os links de download das listagens, e comparar com os links já armazenados talvez

In [2]:
from bs4 import BeautifulSoup

with open('estoques.html') as fp:
    soup = BeautifulSoup(fp, 'html.parser')

print(soup)


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="pt-br" xmlns="http://www.w3.org/1999/xhtml">
<head>
<!-- Arquivo de Cache: |cache/1445,0,0.html| --><!-- não lendo da cache --> <meta content="EdificaPOA - EGLRF" name="description"/>
<meta content="Prefeitura Municipal de Porto Alegre" name="author"/>
<meta content="true" name="MSSmartTagsPreventParsing"/>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="all" name="robots"/>
<meta content="pt-br" name="language"/>
<meta content="EdificaPOA - EGLRF" name="DC.title"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="tags" lang="pt-BR" name="keywords" xml:lang="pt-BR"/>
<meta content="never" name="expires"/>
<meta content="IE=7" http-equiv="X-UA-Compatible"/>
<title>EdificaPOA - EGLRF</title>
<link href="../proweb3_geral/css/estilo_geral.css" rel="stylesheet" title="default" type="text/css"/>
<link href="cs

## Segunda parte: a criação do banco de dados